# 中证500ETF期权策略分析

本笔记本用于分析我们的期权策略：每月初卖出中证500ETF平值看涨期权，到期前7天展期。我们将分析这个策略在一年内的表现，包括收益率、风险指标和交易统计。

In [1]:
# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import sys
import os
from dotenv import load_dotenv

# 加在项目中的.env配置 tushare的token之类的配置文件都可以放在里面
load_dotenv()

# 设置绘图风格
plt.style.use('ggplot')
sns.set(font_scale=1.2)

# 添加项目根目录到路径
sys.path.append('..')

# 导入自定义模块 回测模块的engin 和 数据模块的data_fetcher
from data.data_fetcher import DataFetcher

import matplotlib.font_manager as fm

# 获取所有可用字体
# fonts = fm.findSystemFonts()
# for font in fonts:
#     print(fm.FontProperties(fname=font).get_name())

import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False






## 1. 数据准备

首先，我们需要获取中证500ETF和相关期权的数据。如果数据已经存在，我们将直接加载；否则，我们将从Tushare获取数据。

In [ ]:
# 定义标的种类和日期范围
underlying = 'CSI500ETF'  # 标的种类，这里暂定中证500etf
start_date = '20240101'
end_date = '20241231'

# 创建标的文件夹
underlying_folder = f'../data/{underlying}'
if not os.path.exists(underlying_folder):
    os.makedirs(underlying_folder)

# 生成包含日期的文件名
etf_data_file = f'{underlying_folder}/{start_date}_{end_date}_stock_data.csv'
option_data_file = f'{underlying_folder}/{start_date}_{end_date}_option_data.csv'

if not (os.path.exists(etf_data_file) and os.path.exists(option_data_file)):
    token = os.environ.get('TUSHARE_TOKEN')
    if token is None:
        token = input('请输入Tushare API token: ')

    fetcher = DataFetcher(token=token)

    print(f"正在获取{start_date}至{end_date} {underlying}的数据...")
    etf_data, option_data = fetcher.prepare_backtest_data(start_date, end_date)
    
    # 保存数据
    fetcher.save_data(etf_data, option_data, etf_file=etf_data_file, option_file=option_data_file)
else:
    print("数据文件已存在，将直接加载数据")

## 2. 查看数据

让我们先看看ETF和期权数据的基本情况。

In [5]:
# 加载ETF数据
etf_df = pd.read_csv(etf_data_file)
etf_df['trade_date'] = pd.to_datetime(etf_df['trade_date'])

# 显示ETF数据基本信息
print("ETF数据基本信息：")
print(f"数据时间范围：{etf_df['trade_date'].min()} 至 {etf_df['trade_date'].max()}")
print(f"交易日数量：{len(etf_df)}")
print("\n前5行数据：")
etf_df.head()

NameError: name 'etf_data_file' is not defined

In [ ]:
# 加载期权数据
option_df = pd.read_csv(option_data_file)
option_df['trade_date'] = pd.to_datetime(option_df['trade_date'])
option_df['expire_date'] = pd.to_datetime(option_df['expire_date'])

# 显示期权数据基本信息
print("期权数据基本信息：")
print(f"期权合约数量：{option_df['ts_code'].nunique()}")
print(f"看涨期权数量：{option_df[option_df['call_put'] == 'C']['ts_code'].nunique()}")
print(f"看跌期权数量：{option_df[option_df['call_put'] == 'P']['ts_code'].nunique()}")
print("\n前5行数据：")
option_df.head()

## 3. 可视化ETF价格走势

让我们绘制中证500ETF在回测期间的价格走势图。

In [2]:
plt.figure(figsize=(14, 7))

# 绘制ETF收盘价走势
plt.plot(etf_df['trade_date'], etf_df['close'], label='中证500ETF收盘价')

# 添加标题和标签
plt.title('中证500ETF价格走势', fontsize=15)
plt.xlabel('日期')
plt.ylabel('价格')
plt.grid(True)
plt.legend()

# 优化x轴日期显示
plt.xticks(rotation=45)
plt.tight_layout()


plt.show()

NameError: name 'plt' is not defined

## 4. 运行策略回测

现在，我们将运行我们的期权策略回测，并分析结果。

In [ ]:
from scripts.option.strategies.monthly_atm_call import MonthlyATMCallStrategy

# 初始化策略
strategy = MonthlyATMCallStrategy(
    etf_data=etf_df,
    option_data=option_df,
    initial_capital=1_000_000  # 初始资金100万
)

# 运行回测
strategy.run_backtest()

# 获取结果
results = strategy.get_results()

## 5. 分析回测结果

让我们分析回测结果，包括收益率、交易统计等。

In [ ]:
# 计算关键指标
metrics_df = pd.DataFrame([results['metrics']])

print("策略绩效指标：")
display(metrics_df)

## 6. 可视化交易结果

让我们绘制交易结果的可视化图表。

In [ ]:
# 绘制净值曲线
plt.figure(figsize=(12, 6))
plt.plot(results['nav']['date'], results['nav']['value'])
plt.title('策略净值曲线')
plt.xlabel('日期')
plt.ylabel('净值')
plt.grid(True)
plt.show()